# BioExplorer - Neurons
![](../bioexplorer_neurons_banner.png)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'neurons'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
import libsonata
data_folder = os.getenv('ASTROCYTES_DATA_FOLDER')
print(data_folder)

## Circuit

In [ ]:
import libsonata
circuit_config = os.path.join(data_folder, 'build', 'ngv_config.json')
circuit = libsonata.CircuitConfig.from_file(circuit_config)
population_guid = 1

In [ ]:
import json
json = json.loads(circuit.expanded_json)
json.keys()

In [ ]:
json['connectivities']

### Nodes

In [ ]:
import numpy as np
nodes = circuit.node_population('All')
ids = np.arange(0, nodes.size)

model_templates = nodes.get_attribute('model_template', list(ids))
etypes = nodes.get_attribute('etype', list(ids))
mtypes = nodes.get_attribute('mtype', list(ids))
hyper_columns = nodes.get_attribute('hypercolumn', list(ids))
layers = nodes.get_attribute('layer', list(ids))
morphologies = nodes.get_attribute('morphology', list(ids))
xs = nodes.get_attribute('x', list(ids))
ys = nodes.get_attribute('y', list(ids))
zs = nodes.get_attribute('z', list(ids))
rxs = nodes.get_attribute('rotation_angle_xaxis', list(ids))
rys = nodes.get_attribute('rotation_angle_yaxis', list(ids))
rzs = nodes.get_attribute('rotation_angle_zaxis', list(ids))


In [ ]:
# Morphologies
morphologies_guids = dict()
print('Deleting existing morphologies')
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphology' % db_schema)
    session.commit()

duplicates = list()
print('Populating nodes')
count = 0
with Session(engine) as session:
    for i in tqdm(range(nodes.size)):
        try:
            session.execute(
                'INSERT INTO %s.morphology VALUES (:guid, :basename)' % db_schema,
                {
                    'guid': count,
                    'basename': morphologies[i]
                }
            )
            morphologies_guids[morphologies[i]] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(morphologies[i])
        session.rollback()


In [ ]:
# E-Types
etypes_guids = dict()
print('Deleting existing electrical types')
with Session(engine) as session:
    session.execute('DELETE FROM %s.electrical_type' % db_schema)
    session.commit()

duplicates = list()
print('Populating records')
count = 0
with Session(engine) as session:
    for i in tqdm(range(etypes.size)):
        try:
            session.execute(
                'INSERT INTO %s.electrical_type VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': etypes[i]
                }
            )
            etypes_guids[etypes[i]] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(etypes[i])
        session.rollback()


In [ ]:
# M-Types
mtypes_guids = dict()
print('Deleting existing morphological types')
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphological_type' % db_schema)
    session.commit()

duplicates = list()
print('Populating records')
count = 0
with Session(engine) as session:
    for i in tqdm(range(mtypes.size)):
        try:
            session.execute(
                'INSERT INTO %s.morphological_type VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': mtypes[i]
                }
            )
            mtypes_guids[mtypes[i]] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(mtypes[i])
        session.rollback()


In [ ]:
# Nodes
print('Deleting existing nodes')
with Session(engine) as session:
    session.execute(
        'DELETE FROM %s.node WHERE population_guid=:population_guid' % db_schema,
        {
            'population_guid': population_guid
        }
    )
    session.commit()

errors = list()
print('Populating nodes')
with Session(engine) as session:
    for i in tqdm(range(nodes.size)):
        try:
            session.execute(
                'INSERT INTO %s.node VALUES (:guid, :population_guid, :x, :y, :z, :rotation_x, :rotation_y, :rotation_z, :morphology_guid, :etype, :mtype)' % db_schema,
                {
                    'guid': i,
                    'population_guid': population_guid,
                    'x': xs[i],
                    'y': ys[i],
                    'z': zs[i],
                    'rotation_x': rxs[i],
                    'rotation_y': rys[i],
                    'rotation_z': rzs[i],
                    'morphology_guid': morphologies_guids[morphologies[i]],
                    'etype': etypes_guids[etypes[i]],
                    'mtype': mtypes_guids[mtypes[i]]
                }
            )
            session.commit()
        except Exception as e:
            print(e)
            errors.append(e)
            break
        session.rollback()

## Morphology Sections

In [ ]:
import morphio

components = json['components']
morphology_folder = components['morphologies_dir']

# for morphologies_guid in morphologies_guids:
#     morphology_file = os.path.join(morphology_folder, 'h5', morphologies_guid + '.h5')
#     m = morphio.Morphology(morphology_file)
#     break

In [ ]:
section_types = dict()
section_types[morphio.SectionType.soma] = 0
section_types[morphio.SectionType.axon] = 1
section_types[morphio.SectionType.basal_dendrite] = 2
section_types[morphio.SectionType.apical_dendrite] = 3

In [ ]:
with Session(engine) as session:
    session.execute('DELETE FROM %s.section' % db_schema)
    session.commit()

with Session(engine) as session:
    for morphology_guid in tqdm(range(len(morphologies_guids))):
        morphology=morphologies[morphology_guid]
        morphology_file = os.path.join(morphology_folder, 'h5', morphology + '.h5')
        m = morphio.Morphology(morphology_file)

        for section in m.sections:
            section_type_guid = section_types[section.type]
            parent = -1
            if not section.is_root:
                parent =  section.parent.id

            points = list()
            for i in range(len(section.points)):
                point = section.points[i]
                points.append(point[0])
                points.append(point[1])
                points.append(point[2])
                points.append(section.diameters[i])
            buffer = np.array(points, dtype=np.float32).tobytes()

            point = section.points[0]

            session.execute(
                'INSERT INTO %s.section VALUES (:morphology_guid, :section_guid, :section_parent_guid, :section_type_guid, :points, :x, :y, :z)' % db_schema,
                {
                    'morphology_guid': morphology_guid,
                    'section_guid': section.id,
                    'section_parent_guid': parent,
                    'section_type_guid': section_type_guid,
                    'points': buffer,
                    'x': float(point[0]),
                    'y': float(point[1]),
                    'z': float(point[2])
                }
            )
        if morphology_guid % 10 == 0:
            session.commit()
    session.commit()